In [1]:
A,B,G=0,0,0
isColab=False
if isColab:
    %pip install ultralytics
    import ultralytics
    ultralytics.checks()
    from google.colab.patches import cv2_imshow
    from google.colab import drive
from sqlalchemy.dialects.oracle.dictionary import all_objects
import csv
import pickle
from models.TrackedObject import TrackedObject
from models.VideoAnalysis import VideoAnalysis
from utils.dynamics import EuclideanDistTracker, get_overlap_info
import utils.files as fil
import cv2
import os
import analysis.mask_rcnn as mask_rcnn
from analysis import yolo
from utils.visualise import visualize
%load_ext autoreload
%autoreload 2

from IPython.display import display, clear_output
import PIL.Image

In [2]:

transform, COCO_INSTANCE_CATEGORY_NAMES, model, device= mask_rcnn.config()
#model= yolo.config()

def run_detection(frame_filenames, detection_file_path):
    if os.path.exists(detection_file_path):
        print("Detection file already exists. Skipping YOLO detection.")
        return

    print("Running YOLO detection and saving results...")
    all_detections = []

    for frame_filename in frame_filenames:
        frame = cv2.imread(frame_filename)
        if frame is None:
            all_detections.append([])  # Append empty detection for consistency
            continue

        #detections = yolo.process_frame(model, frame, device, transform,
                     #                   COCO_INSTANCE_CATEGORY_NAMES,
                       #                 tracker=None,
                       #                 conf=0.5)
        detections = mask_rcnn.process_frame(model, frame, device, transform, COCO_INSTANCE_CATEGORY_NAMES, tracker=None, conf=0.5)
        all_detections.append(detections)

    with open(detection_file_path, 'wb') as f:
        pickle.dump(all_detections, f)

    print(f"Saved detections to {detection_file_path}")

def process_clip(frame_filenames, case_folder, detection_file_path):
    acc_detected=False
    tracker = EuclideanDistTracker()
    va=VideoAnalysis()

    if not os.path.exists(detection_file_path):
        raise FileNotFoundError(f"No detection file found at {detection_file_path}")
    with open(detection_file_path, 'rb') as f:
        all_detections = pickle.load(f)


    # Get frame size dynamically from the first frame
    sample_frame = cv2.imread(frame_filenames[0])
    height, width = sample_frame.shape[:2]
    frame_size = (width, height)
    fps = 18

    # Save output to MP4 instead of AVI
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or use 'avc1' for better compatibility
    #out = cv2.VideoWriter(case_folder+"_output.mp4", fourcc, fps, frame_size)

    for idx, frame_filename in enumerate(frame_filenames):
        frame = cv2.imread(frame_filename)
        if frame is None:
            continue
        _objects = all_detections[idx]
        boxes = tracker.update(_objects)
        get_overlap_info(boxes)
        result_frame=visualize(frame, _objects)
        for o in va.allObjects:
          o.is_present=False
          o.is_accident=False
        for obj in _objects:
          if obj.id>(len(va.allObjects)-1):
            obj.H=height
            va.allObjects.append(obj)
          else:
            o=va.allObjects[obj.id]
            o.id=obj.id
            o.is_present=obj.is_present
            o.x1=obj.x1
            o.x2=obj.x2
            o.y1=obj.y1
            o.y2=obj.y2
            o.centroid=obj.centroid
            o.overlaps=obj.overlaps
            o.past_centroids.append(obj.centroid)
            o.compute_speed()
            o.compute_acceleration()
            o.compute_vector()
            #print(str(o.id)+str(o.is_accident)+str(o.centroid))
        get_overlap_info(va.allObjects)
        for box in va.allObjects:
            box.check_accident(A,B,G)
            #cv2.putText(result_frame, str(box.acceleration),(box.x1, box.y1 - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 2)
            if box.is_accident:
              acc_detected=True
              cv2.rectangle(result_frame, (box.x1, box.y1), (box.x2, box.y2), (0, 255, 255), 3)
        #print(frame_filename)
        #out.write(result_frame)
        if isColab:
            a=1
            #Convert to PIL format for proper display in Jupyter/Colab
            #Display only the current frame
            #clear_output(wait=True)

            #result_frame_rgb = cv2.cvtColor(result_frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            #result_pil = PIL.Image.fromarray(result_frame_rgb)
            #display(result_pil)
        else:
            cv2.imshow('res',result_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                  break
    #out.release()
    cv2.destroyAllWindows()
    return acc_detected


Using device: cpu


In [ ]:
if isColab:
    drive.mount('/content/drive/',force_remount=True)
    normal_folder = '/content/drive/MyDrive/data/normal/'
else:
    accident_folder = './data/accident/'
    normal_folder = './data/normal/'

results=[]
for A in range(-20,0):
    folders = os.listdir(normal_folder)
    accident_count, total_count=0,0
    for case_folder in folders:
        frame_filenames= sorted(
            [f for f in os.listdir(normal_folder+case_folder) if f.endswith('.jpg')],
            key=fil.extract_number
        )

        frame_filenames = [os.path.join(normal_folder+case_folder, f) for f in frame_filenames]
        detection_file_path = case_folder + "_detections.pkl"
        if not os.path.exists(detection_file_path):
            run_detection(frame_filenames, detection_file_path)
        else:
            print(f"Detections already exist at {detection_file_path}. Skipping detection.")
        acc_detected = process_clip(frame_filenames, case_folder, detection_file_path)
        if acc_detected == True:
            accident_count+=1
        total_count+=1
    print ("Total count: ",total_count)
    print ("Accident count: ",accident_count)
    print ("A",A,"B",B,"G",G)

    results.append({
        "A": A,
        "total_count": total_count,
        "accident_count": accident_count,
        "B": B,
        "G": G
    })
csv_output_file = "global_stats.csv"
with open(csv_output_file, 'w', newline='') as csvfile:
    fieldnames = ["A", "total_count", "accident_count", "B", "G"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(results)

Mounted at /content/drive/
Running YOLO detection and saving results...
Saved detections to Normal_016.mp4_detections.pkl
Running YOLO detection and saving results...
Saved detections to Normal_006.mp4_detections.pkl
Running YOLO detection and saving results...
Saved detections to Normal_004.mp4_detections.pkl
Running YOLO detection and saving results...
Saved detections to Normal_015.mp4_detections.pkl
Running YOLO detection and saving results...
Saved detections to Normal_007.mp4_detections.pkl
Running YOLO detection and saving results...
Saved detections to Normal_101.mp4_detections.pkl
Running YOLO detection and saving results...
Saved detections to Normal_091.mp4_detections.pkl
Running YOLO detection and saving results...
Saved detections to Normal_090.mp4_detections.pkl
Running YOLO detection and saving results...
Saved detections to Normal_088.mp4_detections.pkl
Running YOLO detection and saving results...
Saved detections to Normal_065.mp4_detections.pkl
Running YOLO detection 

In [4]:
if isColab:
    drive.mount('/content/drive/',force_remount=True)
    normal_folder = '/content/drive/MyDrive/data/normal/'
    accident_folder = '/content/drive/MyDrive/data/accident/'
else:
    accident_folder = './data/accident/'
    normal_folder = './data/normal/'

results=[]
for A in range(-19,0):
    folders = os.listdir(accident_folder)
    accident_count, total_count=0,0
    for case_folder in folders:
        frame_filenames= sorted(
            [f for f in os.listdir(accident_folder+case_folder) if f.endswith('.jpg')],
            key=fil.extract_number
        )

        frame_filenames = [os.path.join(accident_folder+case_folder, f) for f in frame_filenames]
        detection_file_path = case_folder + "_detections.pkl"
        if not os.path.exists(detection_file_path):
            run_detection(frame_filenames, detection_file_path)
        else:
            print(f"Detections already exist at {detection_file_path}. Skipping detection.")
        acc_detected = process_clip(frame_filenames, case_folder, detection_file_path)
        if acc_detected == True:
            accident_count+=1
        total_count+=1
    print ("Total count: ",total_count)
    print ("Accident count: ",accident_count)
    print ("A",A,"B",B,"G",G)

    results.append({
        "A": A,
        "total_count": total_count,
        "accident_count": accident_count,
        "B": B,
        "G": G
    })
csv_output_file = "global_stats.csv"
with open(csv_output_file, 'w', newline='') as csvfile:
    fieldnames = ["A", "total_count", "accident_count", "B", "G"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(results)

Detections already exist at 01_Accident_001.mp4_detections.pkl. Skipping detection.
Detections already exist at 01_Accident_002.mp4_detections.pkl. Skipping detection.
Detections already exist at 01_Accident_004.mp4_detections.pkl. Skipping detection.
Detections already exist at 01_Accident_010.mp4_detections.pkl. Skipping detection.
Detections already exist at 01_Accident_011.mp4_detections.pkl. Skipping detection.
Detections already exist at 01_Accident_013.mp4_detections.pkl. Skipping detection.
Detections already exist at 01_Accident_015.mp4_detections.pkl. Skipping detection.
Detections already exist at 01_Accident_016.mp4_detections.pkl. Skipping detection.
Detections already exist at 01_Accident_017.mp4_detections.pkl. Skipping detection.
Detections already exist at 01_Accident_018.mp4_detections.pkl. Skipping detection.
Detections already exist at 01_Accident_019.mp4_detections.pkl. Skipping detection.
Detections already exist at 01_Accident_020.mp4_detections.pkl. Skipping det